## **loading a 3D model from an OBJ file.**

In [2]:
import numpy as np
from collections import defaultdict
import heapq

def load_obj(file_path):
    vertices = []
    textures = []
    normals = []
    faces = []
    groups = {}
    materials = {}
    current_group = None
    mtl_file = None

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            parts = line.split()
            keyword = parts[0]

            if keyword == 'v':
                vertices.append(list(map(float, parts[1:])))
            elif keyword == 'vt':
                textures.append(list(map(float, parts[1:])))
            elif keyword == 'vn':
                normals.append(list(map(float, parts[1:])))
            elif keyword == 'f':
                face_data = []
                for vertex_part in parts[1:]:
                    indices = vertex_part.split('/')
                    v_index = int(indices[0])
                    if v_index < 0:
                        v_index += len(vertices)
                    v_index -= 1

                    vt_index = None
                    if len(indices) > 1 and indices[1]:
                        vt_index = int(indices[1])
                        if vt_index < 0:
                            vt_index += len(textures)
                        vt_index -= 1

                    vn_index = None
                    if len(indices) > 2 and indices[2]:
                        vn_index = int(indices[2])
                        if vn_index < 0:
                            vn_index += len(normals)
                        vn_index -= 1

                    face_data.append((v_index, vt_index, vn_index))
                faces.append(face_data)
                if current_group:
                    if current_group not in groups:
                        groups[current_group] = []
                    groups[current_group].append(len(faces) - 1) # Stocker l'index de la face
            elif keyword == 'g':
                if len(parts) > 1:
                    current_group = parts[1]
            elif keyword == 'usemtl':
                if len(parts) > 1:
                    current_material = parts[1]
                    # Vous pouvez choisir de stocker l'association face -> matériau si nécessaire
            elif keyword == 'mtllib':
                if len(parts) > 1:
                    mtl_file = parts[1]
                    # Vous pouvez ajouter une fonction pour parser le fichier .mtl ici

    return vertices, textures, normals, faces, groups, materials, mtl_file

## **simplify the mesh by grouping nearby vertices into cells of a 3D grid and replacing them with a single representative vertex**

In [3]:
def vertex_clustering(vertices, textures, normals, faces, grid_resolution):
    if isinstance(grid_resolution, int):
        grid_resolution = [grid_resolution] * 3

    min_coords = np.min(vertices, axis=0)
    max_coords = np.max(vertices, axis=0)
    grid_size = (max_coords - min_coords) / np.array(grid_resolution)

    # 2. Assigner les sommets aux cellules
    cell_map = defaultdict(list)
    for i, vertex in enumerate(vertices):
        cell_index = tuple(((vertex - min_coords) // grid_size).astype(int))
        cell_map[cell_index].append(i)

    # 3. Calculer le barycentre, la texture moyenne et la normale moyenne pour chaque cellule
    new_vertices = []
    new_textures = []
    new_normals = []
    cell_barycenter_map = {}  # Associe l'index de la cellule à l'index du nouveau sommet
    for cell_index, vertex_indices in cell_map.items():
        cell_vertices = [vertices[i] for i in vertex_indices]
        if cell_vertices:
            barycenter = np.mean(cell_vertices, axis=0).tolist()
            new_vertices.append(barycenter)
            cell_barycenter_map[cell_index] = len(new_vertices) - 1

            # Calculer la texture moyenne
            cell_textures = [textures[i] for i in vertex_indices if textures]
            if cell_textures:
                avg_texture = np.mean(cell_textures, axis=0).tolist()
                new_textures.append(avg_texture)
            else:
                new_textures.append(None)  # Gérer le cas sans textures

            # Calculer la normale moyenne
            cell_normals = [normals[i] for i in vertex_indices if normals]
            if cell_normals:
                avg_normal = np.mean(cell_normals, axis=0).tolist()
                new_normals.append(avg_normal)
            else:
                new_normals.append(None)  # Gérer le cas sans normales

    # 4. Mettre à jour les faces
    new_faces = []
    for face in faces:
        new_face_data = []
        valid_face = True
        for vertex_index, texture_index, normal_index in face:
            vertex = vertices[vertex_index]
            cell_index = tuple(((vertex - min_coords) // grid_size).astype(int))
            if cell_index in cell_barycenter_map:
                new_vertex_index = cell_barycenter_map[cell_index]

                # Trouver l'index de texture et de normale correspondant au barycentre de la cellule
                new_texture_index = new_vertex_index if new_textures[new_vertex_index] is not None else None
                new_normal_index = new_vertex_index if new_normals[new_vertex_index] is not None else None

                new_face_data.append((new_vertex_index, new_texture_index, new_normal_index))
            else:
                valid_face = False # Skip faces where original vertices are not assigned to any cell
                break

        if valid_face:
            # Éviter les faces dégénérées
            unique_vertices_in_face = list(set(idx for idx, _, _ in new_face_data))
            if len(unique_vertices_in_face) >= 3:
                new_faces.append(new_face_data)

    # Filter out None values from new_textures and new_normals
    new_textures_filtered = [t for t in new_textures if t is not None]
    new_normals_filtered = [n for n in new_normals if n is not None]

    # Re-index faces to match the filtered texture and normal lists
    if new_textures_filtered or new_normals_filtered:
        new_faces_reindexed = []
        for face in new_faces:
            new_face_data_reindexed = []
            for v_idx, vt_idx, vn_idx in face:
                original_vt_index = new_textures.index(new_textures_filtered[vt_idx]) if vt_idx is not None and new_textures_filtered else None
                original_vn_index = new_normals.index(new_normals_filtered[vn_idx]) if vn_idx is not None and new_normals_filtered else None
                new_face_data_reindexed.append((v_idx, original_vt_index, original_vn_index))
            new_faces_reindexed.append(new_face_data_reindexed)
        return new_vertices, new_textures_filtered, new_normals_filtered, new_faces_reindexed
    else:
        return new_vertices, [], [], new_faces

### 3. Extracts unique edges from the list of faces. An edge is defined by a pair of connected vertices.

### 4. Calculates a cost for collapsing a specific edge. In this case, the cost is simply the geometric distance between the two vertices forming the edge.
    
### 5. Calculates the cost for each edge in a set of edges and stores them in a min-heap data structure. A min-heap allows efficient retrieval of the edge with the lowest cost.

### 6. Performs the core operation of edge collapse, merging the two vertices of a selected edge into a single vertex.

In [4]:
def get_edges_from_faces(faces):
    """
    Extracts unique edges from a list of faces.

    Args:
        faces (list): List of faces, where each face is a list of vertex indices.

    Returns:
        set: A set of tuples, where each tuple represents an edge (sorted vertex indices).
    """
    edges = set()
    for face in faces:
        v_indices = sorted([face[i][0] for i in range(len(face))])
        for i in range(len(v_indices)):
            for j in range(i + 1, len(v_indices)):
                edges.add(tuple(sorted((v_indices[i], v_indices[j]))))
    return edges

def calculate_edge_cost(vertices, edge):
    """
    Calculates the geometric distance between the vertices of an edge.

    Args:
        vertices (list): List of vertex coordinates.
        edge (tuple): Tuple representing an edge (vertex indices).

    Returns:
        float: The geometric distance between the vertices.
    """
    v1_index, v2_index = edge
    v1_coords = np.array(vertices[v1_index])
    v2_coords = np.array(vertices[v2_index])
    return np.linalg.norm(v1_coords - v2_coords)

def initialize_edge_costs(vertices, edges):
    """
    Calculates the cost for each edge and stores them in a min-heap.

    Args:
        vertices (list): List of vertex coordinates.
        edges (set): Set of edges.

    Returns:
        list: A min-heap of (cost, edge) tuples.
    """
    edge_heap = []
    for edge in edges:
        cost = calculate_edge_cost(vertices, edge)
        heapq.heappush(edge_heap, (cost, edge))
    return edge_heap

def collapse_edge(vertices, textures, normals, faces, edge_to_collapse, collapse_to='midpoint'):
    """
    Collapses a given edge, merging its two vertices.

    Args:
        vertices (list): List of vertex coordinates.
        textures (list): List of texture coordinates.
        normals (list): List of normal vectors.
        faces (list): List of faces.
        edge_to_collapse (tuple): The edge to collapse (vertex indices).
        collapse_to (str): How to calculate the new vertex position ('midpoint', 'v1', 'v2').

    Returns:
        tuple: Updated vertices, textures, normals, and faces.
    """
    v1_index, v2_index = sorted(edge_to_collapse)
    v1 = np.array(vertices[v1_index])
    v2 = np.array(vertices[v2_index])

    # Calculate the new vertex position
    if collapse_to == 'midpoint':
        new_vertex_pos = ((v1 + v2) / 2).tolist()
    elif collapse_to == 'v1':
        new_vertex_pos = v1.tolist()
    elif collapse_to == 'v2':
        new_vertex_pos = v2.tolist()
    else:
        raise ValueError("Invalid collapse_to method.")

    # Add the new vertex
    new_vertex_index = len(vertices)
    new_vertices = vertices
    if v1_index < len(new_vertices):
        new_vertices[v1_index] = new_vertex_pos
    else:
        new_vertices.append(new_vertex_pos)

    new_faces = []
    for face in faces:
        new_face = []
        replace_v1 = False
        replace_v2 = False
        for v_idx, vt_idx, vn_idx in face:
            if v_idx == v1_index:
                replace_v1 = True
                new_face.append((v2_index, vt_idx, vn_idx))
            elif v_idx == v2_index:
                replace_v2 = True
                new_face.append((v2_index, vt_idx, vn_idx))
            else:
                new_face.append((v_idx if v_idx < v1_index else v_idx -1 if v_idx > v1_index else v_idx, vt_idx, vn_idx))

        unique_vertices = set(v_idx for v_idx, _, _ in new_face)
        if len(unique_vertices) >= 3:
            new_faces.append(new_face)

    # Remove the collapsed vertex. Adjust indices in faces.
    # new_vertices = [v for i, v in enumerate(vertices) if i != v1_index]
    # new_faces = []
    # for face in faces:
    #     if all(idx[0] != v1_index for idx in face):
    #         remapped_face = []
    #         for v_idx, vt_idx, vn_idx in face:
    #             new_v_idx = v_idx if v_idx < v1_index else v_idx - 1
    #             remapped_face.append((new_v_idx, vt_idx, vn_idx))
    #         new_faces.append(remapped_face)

    return new_vertices, textures, normals, new_faces

### 7. This function implements the iterative edge collapse simplification algorithm. It repeatedly collapses the "cheapest" edge (the one with the smallest cost) until a target number of edge collapses is reached.

In [5]:
def simplify_mesh(vertices, textures, normals, faces, target_reduction):
    """
    Simplifies a mesh by iteratively collapsing edges.

    Args:
        vertices (list): List of vertex coordinates.
        textures (list): List of texture coordinates.
        normals (list): List of normal vectors.
        faces (list): List of faces.
        target_reduction (int): The target number of edge collapses.

    Returns:
        tuple: Simplified vertices, textures, normals, and faces.
    """
    current_vertices = list(vertices)
    current_textures = list(textures)
    current_normals = list(normals)
    current_faces = list(faces)

    edges = get_edges_from_faces(current_faces)
    edge_heap = initialize_edge_costs(current_vertices, edges)

    collapsed_edges_count = 0
    while collapsed_edges_count < target_reduction and edge_heap:
        cost, edge_to_collapse = heapq.heappop(edge_heap)

        # Check if the edge is still valid (vertices haven't been merged yet)
        if edge_to_collapse[0] >= len(current_vertices) or edge_to_collapse[1] >= len(current_vertices) or edge_to_collapse[0] == edge_to_collapse[1]:
            continue

        current_vertices, current_textures, current_normals, current_faces = collapse_edge(
            current_vertices, current_textures, current_normals, current_faces, edge_to_collapse
        )
        collapsed_edges_count += 1

        # Rebuild the edge heap (can be optimized)
        edges = get_edges_from_faces(current_faces)
        edge_heap = initialize_edge_costs(current_vertices, edges)

    return current_vertices, current_textures, current_normals, current_faces

# **Test**

In [6]:
vertices, textures, normals, faces, groups, materials, mtl_file = load_obj("/kaggle/input/objects/obj1.obj")


In [7]:
new_vertices_vc, new_textures_vc, new_normals_vc, new_faces_vc = vertex_clustering(
    vertices, textures, normals, faces, (5, 10, 15)
)

print("Nombre de sommets avant le clustering:", len(vertices))
print("Nombre de sommets après le clustering:", len(new_vertices_vc))
print("Nombre de textures avant le clustering:", len(textures))
print("Nombre de textures après le clustering:", len(new_textures_vc))
print("Nombre de normales avant le clustering:", len(normals))
print("Nombre de normales après le clustering:", len(new_normals_vc))
print("Nombre de faces avant le clustering:", len(faces))
print("Nombre de faces après le clustering:", len(new_faces_vc))

Nombre de sommets avant le clustering: 1369
Nombre de sommets après le clustering: 270
Nombre de textures avant le clustering: 0
Nombre de textures après le clustering: 0
Nombre de normales avant le clustering: 1393
Nombre de normales après le clustering: 270
Nombre de faces avant le clustering: 2734
Nombre de faces après le clustering: 634


In [8]:
# Get edges and initialize edge costs
edges = get_edges_from_faces(new_faces_vc)
print(f"Number of edges: {len(edges)}")
edge_heap = initialize_edge_costs(new_vertices_vc, edges)
print(f"Number of edges in heap: {len(edge_heap)}")

Number of edges: 867
Number of edges in heap: 867


In [9]:
# Simplify the mesh
target_reduction = 100  # Define the number of edges to collapse
simplified_vertices, simplified_textures, simplified_normals, simplified_faces = simplify_mesh(
    new_vertices_vc, new_textures_vc, new_normals_vc, new_faces_vc, target_reduction
)

print("Nombre de sommets après simplification:", len(simplified_vertices))
print("Nombre de faces après simplification:", len(simplified_faces))

Nombre de sommets après simplification: 270
Nombre de faces après simplification: 266


In [10]:
pip install pyvista

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pyvista as pv
import numpy as np
from IPython.display import Image, display

# --- Visualization Code ---
def visualize_mesh(vertices, faces, title, filename):
    """Visualizes a 3D mesh using PyVista and saves the output to a PNG file."""
    if not vertices or not faces:
        print(f"Cannot visualize: {title} has no vertices or faces.")
        return

    # Convert face data to the format PyVista expects
    pyvista_faces = []
    for face in faces:
        vertex_indices = [f[0] for f in face]  # Extract vertex indices
        pyvista_faces.extend([len(vertex_indices)] + vertex_indices)
    pyvista_faces = np.array(pyvista_faces)

    mesh = pv.PolyData(vertices, pyvista_faces)
    plotter = pv.Plotter(off_screen=True)  # Use off_screen=True for headless rendering
    plotter.add_mesh(mesh, color='lightblue', opacity=0.8, edge_color='black', line_width=0.5)
    plotter.add_text(title, position='upper_left', font_size=14)
    plotter.show(screenshot=filename)  # Save visualization to file

    # Display the saved image
    display(Image(filename=filename))

# Example usage:
# Visualize the object after mesh simplification
visualize_mesh(
    simplified_vertices,
    simplified_faces,
    f"Object after Mesh Simplification (Target Reduction: {target_reduction})",
    "simplified_mesh.png"
)

# --- Optional: Visualize all three side-by-side ---
if vertices and faces and new_vertices_vc and new_faces_vc and simplified_vertices and simplified_faces:
    p = pv.Plotter(shape=(1, 3), window_size=[1600, 600], off_screen=True)

    # Original Object
    p.subplot(0, 0)
    original_mesh_pv_faces = []
    for face in faces:
        vertex_indices = [f[0] for f in face]
        original_mesh_pv_faces.extend([len(vertex_indices)] + vertex_indices)
    original_mesh = pv.PolyData(vertices, np.array(original_mesh_pv_faces))
    p.add_mesh(original_mesh, color='lightblue', opacity=0.8, edge_color='black', line_width=0.5)
    p.add_text("Original", position='upper_left', font_size=12)

    # Vertex Clustered Object
    p.subplot(0, 1)
    vc_mesh_pv_faces = []
    for face in new_faces_vc:
        vertex_indices = [f[0] for f in face]
        vc_mesh_pv_faces.extend([len(vertex_indices)] + vertex_indices)
    vc_mesh = pv.PolyData(new_vertices_vc, np.array(vc_mesh_pv_faces))
    p.add_mesh(vc_mesh, color='lightgreen', opacity=0.8, edge_color='black', line_width=0.5)
    p.add_text("Vertex Clustering", position='upper_left', font_size=12)

    # Simplified Object
    p.subplot(0, 2)
    simplified_mesh_pv_faces = []
    for face in simplified_faces:
        vertex_indices = [f[0] for f in face]
        simplified_mesh_pv_faces.extend([len(vertex_indices)] + vertex_indices)
    simplified_mesh = pv.PolyData(simplified_vertices, np.array(simplified_mesh_pv_faces))
    p.add_mesh(simplified_mesh, color='lightcoral', opacity=0.8, edge_color='black', line_width=0.5)
    p.add_text(f"Mesh Simplification (Reduction: {target_reduction})", position='upper_left', font_size=12)

    p.show(screenshot="all_meshes.png")  # Save visualization to file

    # Display the saved image
    display(Image(filename="all_meshes.png"))
